# SOC 210B Assignment 7
_By M. Nolan Gray_

In [75]:
import statsmodels.api as sm
import pandas as pd
from statsmodels.formula.api import logit
import numpy as np

In [85]:
dtafile = './Data/nlsw88_occ4.dta'
df = pd.read_stata(dtafile)
df.head(5)

,idcode,age,race,married,never_married,grade,collgrad,south,smsa,c_city,industry,occupation,union,wage,hours,ttl_exp,tenure,occ4
0,1,37,black,single,0,12.0,not college grad,0,SMSA,0,Transport/Comm/Utility,Operatives,union,11.739125,48.0,10.333334,5.333333,upper manual
1,2,37,black,single,0,12.0,not college grad,0,SMSA,1,Manufacturing,Craftsmen,union,6.400963,40.0,13.621795,5.250000,upper manual
2,3,42,black,single,1,12.0,not college grad,0,SMSA,1,Manufacturing,Sales,NaN,5.016723,40.0,17.730770,1.250000,lower nonmanual
3,4,43,white,married,0,17.0,college grad,0,SMSA,0,Professional Services,Other,union,9.033813,42.0,13.211537,1.750000,other
4,6,42,white,married,0,12.0,not college grad,0,SMSA,0,Manufacturing,Operatives,nonunion,8.083731,48.0,17.820513,17.750000,upper manual


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2246 entries, 0 to 2245
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   idcode         2246 non-null   int16   
 1   age            2246 non-null   int8    
 2   race           2246 non-null   category
 3   married        2246 non-null   category
 4   never_married  2246 non-null   int8    
 5   grade          2244 non-null   float64 
 6   collgrad       2246 non-null   category
 7   south          2246 non-null   int8    
 8   smsa           2246 non-null   category
 9   c_city         2246 non-null   int8    
 10  industry       2232 non-null   category
 11  occupation     2237 non-null   category
 12  union          1878 non-null   category
 13  wage           2246 non-null   float32 
 14  hours          2242 non-null   float64 
 15  ttl_exp        2246 non-null   float32 
 16  tenure         2231 non-null   float32 
 17  occ4           2237 non-null   ca

## Question 1

In [18]:
formula = ('never_married ~ collgrad + race')
model = logit(formula = formula, data = df).fit()

Optimization terminated successfully.
         Current function value: 0.319399
         Iterations 7


In [19]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          never_married   No. Observations:                 2246
Model:                          Logit   Df Residuals:                     2242
Method:                           MLE   Df Model:                            3
Date:                Fri, 04 Mar 2022   Pseudo R-squ.:                 0.04424
Time:                        08:01:22   Log-Likelihood:                -717.37
converged:                       True   LL-Null:                       -750.57
Covariance Type:            nonrobust   LLR p-value:                 2.506e-14
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -2.7171      0.110    -24.710      0.000      -2.933      -2.502
collgrad[T.college grad]     0.7287      0.151      4.815      0.000       0.432       1.025
race[T.black]                1.0477      0.145      7.243      0.000       0.764       1.331
race[T.other]                0.3804      0.625      0.608      0.543      -0.845       1.606
============================================================================================
"""

Our log-odds formula is this: -2.72 + 0.73(college) + 1.05(black). I have left out race:other as it is not statistically significant. The default is a white woman with no college degree.

A white woman with a college degree:
* Log-odds: -1.99 (-2.72 + 0.73)
* Odds: 0.14 (e^-1.99/(1+e^-1.99)) 
* Probability: 0.12 (0.14/(1+0.14))

A black woman with a college degree:
* Log-odds: -0.94 (-2.72 + 0.73 + 1.05)
* Odds: 0.39 (e^-0.94/(1+e^-0.94))
* Probability: 0.28 (0.39/(1+0.39))

## Question 2

In [77]:
x1 = df[['collgrad','race']]
y1 = df['occ4']

In [78]:
x1 = sm.add_constant(x1, prepend = False)

In [86]:
list1= ['upper nonmanual', 'lower nonmanual', 'upper manual', 'lower manual', 'other']

In [87]:
mapping_dict = {}
for index, i in enumerate(list1):
    mapping_dict[i] = index

mapping_dict

{'upper nonmanual': 0,
 'lower nonmanual': 1,
 'upper manual': 2,
 'lower manual': 3,
 'other': 4}

In [88]:
x_dummy = pd.get_dummies(x)
y_dummy = y.map(mapping_dict)

In [89]:
x_dummy

,const,collgrad_not college grad,collgrad_college grad,race_white,race_black,race_other
0,1.0,1,0,0,1,0
1,1.0,1,0,0,1,0
2,1.0,1,0,0,1,0
3,1.0,0,1,1,0,0
4,1.0,1,0,1,0,0
...,...,...,...,...,...,...
2241,1.0,1,0,1,0,0
2242,1.0,0,1,1,0,0
2243,1.0,1,0,1,0,0
2244,1.0,1,0,0,1,0


In [90]:
y_dummy

0       2
1       2
2       1
3       4
4       2
       ..
2241    3
2242    0
2243    1
2244    3
2245    1
Name: occ4, Length: 2246, dtype: category
Categories (5, int64): [0 < 1 < 2 < 3 < 4]

In [105]:
# can't figure out how to specify reference cell as 1!
mnlogit_mod = sm.MNLogit(y_dummy, x_dummy)
mnlogit_fit = mnlogit_mod.fit()

print(mnlogit_fit.summary())

         Current function value: 1.292898
         Iterations: 35


/opt/conda/lib/python3.9/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                          MNLogit Regression Results                          
Dep. Variable:                   occ4   No. Observations:                 2246
Model:                        MNLogit   Df Residuals:                     2226
Method:                           MLE   Df Model:                           16
Date:                Fri, 04 Mar 2022   Pseudo R-squ.:                  0.1274
Time:                        20:44:59   Log-Likelihood:                -2903.8
converged:                      False   LL-Null:                       -3327.7
Covariance Type:            nonrobust   LLR p-value:                4.107e-170
                   occ4=1       coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -0.0306   1.57e+06  -1.94e-08      1.000   -3.08e+06    3.08e+06
collgrad_not college grad     0.7161        nan        nan        nan         nan     

/opt/conda/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:4236: RuntimeWarning: invalid value encountered in sqrt
  bse = np.sqrt(np.diag(self.cov_params()))


A black college grade has an upper nonmanual job relative to a lower nonmanual job:

* Log-odds: * (*)
* Odds: * (e^*/(1+e^*))
* Probability: * (*/(1+*))

A black woman with a college degree has a lower manual job relative to a lower nonmanual job:

* Log-odds: * (*)
* Odds: * (e^*/(1+e^*))
* Probability: (*/(1+*))